In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import natsort
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob

# data 1 : URFD
- step 1 : pack each directory containing .png files as .avi file (to follow UCF format)
- step 2 : re-arange video files according to the class label
- step 3 : copy-and-paste UCF clip generation code...

In [ ]:
root = '/data/FallDownData/URFD_origin/raw'
video_files = natsort.natsorted(glob.glob(root + '/*/*'))
for video in tqdm(video_files):
    prefix, ext = os.path.splitext(video)
    save_path = prefix.replace('raw', 'video') + '.avi'
    
    frame_list = natsort.natsorted(glob.glob(video+'/*'))        
    first_frame = cv2.imread(frame_list[0])
    out = cv2.VideoWriter(save_path,
                          cv2.VideoWriter_fourcc('M','J','P','G'), 
                          30, (first_frame.shape[:-1][::-1]))
    for img_file in frame_list:
        img_arr = cv2.imread(img_file)
        out.write(img_arr)
        
    os.system('mkdir -p {}'.format(os.path.dirname(save_path)))
    out.release()

In [ ]:
# root = '/data/FallDownData/URFD/raw'
# video_files = natsort.natsorted(glob.glob(root + '/*/*'))
# for video in tqdm(video_files):
#     prefix, ext = os.path.splitext(video)
#     save_path = prefix.replace('raw', 'video') + '.avi'
    
#     frames = []
    
#     cap = cv2.VideoCapture(video)
#     frame_height, frame_width = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    
#     out = cv2.VideoWriter(save_path,
#                           cv2.VideoWriter_fourcc('M','J','P','G'), 
#                           30, (frame_width//2,frame_height))
    
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break
            
#         # left : depth, right : color
#         color_frame = frame[:, frame_width//2:]
#         out.write(color_frame)

#     os.system('mkdir -p {}'.format(os.path.dirname(save_path)))
#     out.release()

# data 2 : Multicam FDD
- this dataset is bullshit!
- syncronization is not matched well

In [42]:
root = '/data/FallDownData/MulticamFD/raw'
df = pd.read_csv(os.path.join(os.path.dirname(root), 'Multicam_Annotations.csv'))

In [43]:
df.head()

,id,camera_reference,start,end,code
0,1,11,874,1011,1
1,1,11,1012,1079,6
2,1,11,1080,1108,2
3,1,11,1109,1285,3
4,2,4,308,374,1


In [45]:
df[df.id==1]

,id,camera_reference,start,end,code
0,1,11,874,1011,1
1,1,11,1012,1079,6
2,1,11,1080,1108,2
3,1,11,1109,1285,3


In [ ]:
def arange_multicam_dataset(root):
    videos = []

    for dirpath, dirnames, filenames in os.walk(root):
        for fn in filenames:
            name, ext = os.path.splitext(fn)
            if ext == '.avi':
                videos.append(os.path.join(dirpath, fn))


    videos = natsort.natsorted(videos)
    scenarios = np.unique([os.path.dirname(p) for p in videos])
    
    for scenario in tqdm(scenarios):
        def get_shortest_frame_count():

            shortest_frame_count = np.inf

            for i in range(1,9):
                vpath = os.path.join(scenario, f'cam{i}.avi')
                cap = cv2.VideoCapture(vpath)
                frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
                if frame_count < shortest_frame_count:
                    shortest_frame_count = frame_count

            shortest_frame_count = int(shortest_frame_count)

            return shortest_frame_count

        shortest_frame_count = get_shortest_frame_count()

        def get_syncronized_videos():
            res = []
            for i in range(1,9):
                vpath = os.path.join(scenario, f'cam{i}.avi')
                cap = cv2.VideoCapture(vpath)
                frames = []
                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    frames.append(frame)

                res.append(frames[-shortest_frame_count:])

            return np.array(res)

        syncronized = get_syncronized_videos()

        scenario_id = eval(os.path.basename(scenario).split('chute')[1].lstrip('0'))
        scenario_video_home = os.path.dirname(scenario).replace('raw', 'video')

        for seg_ix, (start, end, code) in enumerate(df[df.id==scenario_id][["start","end","code"]].values):
            segment = syncronized[:, start:end]  # a frame segment

            label = 'fall' if code == 2 else 'adl'
            scenario_video = os.path.join(scenario_video_home, label)

            os.system('mkdir -p {}'.format(scenario_video))

            for i in range(1,9):
                seg_ix_str = str(seg_ix+1).zfill(2)
                vpath = os.path.join(scenario, f'cam{i}.avi')
                scenario_indicator = os.path.basename(scenario)

                seg_vpath = os.path.join(scenario_video, f'{scenario_indicator}-cam{i}-s{seg_ix_str}.avi')

                frame_height, frame_width = segment[i-1][0].shape[:2]

                fps = cv2.VideoCapture(vpath).get(cv2.CAP_PROP_FPS)

                out = cv2.VideoWriter(seg_vpath,
                                  cv2.VideoWriter_fourcc('M','J','P','G'), 
                                  fps, (frame_width,frame_height))

                for frame in segment[i-1]:
                    out.write(frame)

                    
arange_multicam_dataset(root)

In [ ]:
cap = cv2.VideoCapture('/data/FallDownData/MulticamFD/video/fall/chute09-cam6-s05.avi')

frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

def check_detection_result(img, box):
    #bx,by,bw,bh = 307.466186523,196.785614014,86.6007080078,166.478393555
    bx,by,bw,bh = box
    xmin, ymin, xmax, ymax = int(bx-bw/2), int(by-bh/2), int(bx+bw/2), int(by+bh/2)
    return img[ymin:ymax, xmin:xmax]

plt.imshow(check_detection_result(frames[16], (370.211120605,174.12197876,88.2487030029,67.7461776733)))

In [ ]:
plt.imshow(frames[17])

In [ ]:
cap.get(cv2.CAP_PROP_FPS)

In [ ]:
with open('/data/torch_data/UCF-101/ucfTrainTestlist/trainlist01.txt') as f:
    ucfList = f.readlines()
    labels = [ line.strip().split()[1] for line in ucfList ]

In [ ]:
np.unique(labels, return_counts=True)

# Create Traintests-plit for cross-validation

In [2]:
from sklearn.model_selection import StratifiedKFold, LeaveOneOut
import itertools

In [3]:
def create_traintestList(root, annotation_path, random_state=0, n_splits=5):
    dataset_name = os.path.basename(os.path.dirname(root.rstrip('/')))
    
    os.system(f'mkdir -p {annotation_path}')

    video_dirs = natsort.natsorted(glob.glob(root + '/*/*'))

    np.random.seed(random_state)
    np.random.shuffle(video_dirs)

    class2idx = {'adl':0, 'fall':1}

    formated_video_dirs = np.array([ x[len(root.rstrip('/'))+1:] for x in video_dirs ])
    
    # split based on scenario
#     scenarios = np.array(
#         list(set([ os.path.basename(x).split('-')[0] for x in formated_video_dirs ]))
#     )

    # split based on camera
    scenarios = np.array(
        natsort.natsorted(list(set([ os.path.basename(x).split('-')[1] for x in formated_video_dirs ]))
    ))

    loo = LeaveOneOut()
    kf = StratifiedKFold(n_splits=n_splits)
    
    data_to_split = None
    if dataset_name == 'URFD':
        data_to_split = formated_video_dirs
    elif dataset_name == 'MulticamFD':
        data_to_split = scenarios    
        
    def get_train_test_split(train_ix, test_ix):
        if dataset_name == 'URFD':
            _train, _test = np.array(formated_video_dirs)[train_ix], np.array(formated_video_dirs)[test_ix]
        elif dataset_name == 'MulticamFD':
            train_scenario_str, test_scenario_str = scenarios[train_ix], scenarios[test_ix]
            print(f'[splist-{k}] train : {np.unique(train_scenario_str)}, test : {np.unique(test_scenario_str)}')
            
            _test = []
            for s in test_scenario_str:
                _test.append(formated_video_dirs[np.char.find(formated_video_dirs, s) != -1])

                
            _test = list(itertools.chain(*_test))
            _train = list(set(formated_video_dirs)-set(_test))

        return _train, _test


    for k, (train_ix, test_ix) in enumerate(kf.split(formated_video_dirs, [os.path.dirname(x) for x in formated_video_dirs])):
#     for k, (train_ix, test_ix) in enumerate(loo.split(scenarios)):
        print()

#         _train, _test = get_train_test_split(train_ix, test_ix)
        _train, _test = np.array(formated_video_dirs)[train_ix], np.array(formated_video_dirs)[test_ix]
    
        _train_lab = [ os.path.dirname(x) for x in _train ]
        _test_lab = [ os.path.dirname(x) for x in _test ]
        print()
        print(f'[splist-{k}] train : {np.unique(_train_lab, return_counts=True)}, test : {np.unique(_test_lab, return_counts=True)}')
        for _split, _data in zip(['train', 'test'], [_train, _test]):
            lines = []
            for i in range(len(_data)):
                line = os.path.splitext(_data[i])[0] + " "
                line += str(class2idx[os.path.dirname(_data[i])])

                lines.append(line + '\n')

            with open(os.path.join(annotation_path, f"{_split}list{k+1:02d}.txt"), 'w') as fp:
                fp.writelines(lines)

In [4]:
# URFD
create_traintestList(root = '/data/FallDownData/URFD/video',
                     annotation_path = '/data/FallDownData/URFD/TrainTestlist',
                     n_splits=5)



[splist-0] train : (array(['adl', 'fall'], dtype='<U4'), array([32, 47])), test : (array(['adl', 'fall'], dtype='<U4'), array([ 8, 12]))


[splist-1] train : (array(['adl', 'fall'], dtype='<U4'), array([32, 47])), test : (array(['adl', 'fall'], dtype='<U4'), array([ 8, 12]))


[splist-2] train : (array(['adl', 'fall'], dtype='<U4'), array([32, 47])), test : (array(['adl', 'fall'], dtype='<U4'), array([ 8, 12]))


[splist-3] train : (array(['adl', 'fall'], dtype='<U4'), array([32, 47])), test : (array(['adl', 'fall'], dtype='<U4'), array([ 8, 12]))


[splist-4] train : (array(['adl', 'fall'], dtype='<U4'), array([32, 48])), test : (array(['adl', 'fall'], dtype='<U4'), array([ 8, 11]))


In [5]:
# MulticamFD
create_traintestList(root = '/data/FallDownData/MulticamFD/video',
                     annotation_path = '/data/FallDownData/MulticamFD/TrainTestlist')



[splist-0] train : (array(['adl', 'fall'], dtype='<U4'), array([582, 147])), test : (array(['adl', 'fall'], dtype='<U4'), array([146,  37]))


[splist-1] train : (array(['adl', 'fall'], dtype='<U4'), array([582, 147])), test : (array(['adl', 'fall'], dtype='<U4'), array([146,  37]))


[splist-2] train : (array(['adl', 'fall'], dtype='<U4'), array([582, 148])), test : (array(['adl', 'fall'], dtype='<U4'), array([146,  36]))


[splist-3] train : (array(['adl', 'fall'], dtype='<U4'), array([583, 147])), test : (array(['adl', 'fall'], dtype='<U4'), array([145,  37]))


[splist-4] train : (array(['adl', 'fall'], dtype='<U4'), array([583, 147])), test : (array(['adl', 'fall'], dtype='<U4'), array([145,  37]))


In [ ]:
LeaveOneOut()